In [2]:
"""Calculate motif enrichment from sample data."""


import numpy as np
import scipy
from scipy.stats.distributions import binom


FOREGROUND_DICT = {"UGAUUC": 5, "UAAACC": 3, "AAGUUACCU": 1,
                   "AAGCCUU": 1, "AGUUCUA": 1, "UUUCCCG": 5}
BACKGROUND_DICT = {"UGAUUC": 3, "UAAACC": 5, "AAGCCUUAU": 1,
                   "AGUUCUA": 1, "UUUCCCG": 5, "UUGGAA": 7}

Calculates enrichment and p-values of motifs with similar lengths.

Args:
    foreground: dictionary of motifs with counts
    background: dictionary of motifs with counts

Returns:
    Dictionary including motifs, enrichment score and p-value

Example: 
    foreground = {"UGAUUC": 5, "UAAACC": 3, "AAGUUACCU": 1, "AAGCCUU": 1, "AGUUCUA": 1, "UUUCCCG": 5},
    background = {"UGAUUC": 3, "UAAACC": 5, "AAGCCUUAU": 1, "AGUUCUA": 1, "UUUCCCG": 5, "UUGGAA": 7}
    
    >>> motifEnrichment(foreground, background)
        {'UGAUUC': [0.7333333333333334, 0.01646090534979423],
        'UAAACC': [0.7333333333333334, 0.21947873799725662],
        'AAGUUACCU': [1.375, 0.26337448559670795],
        'AAGCCUU': [1.6041666666666667, 0.26337448559670795],
        'AGUUCUA': [1.6041666666666667, 0.26337448559670795],
        'UUUCCCG': [1.6041666666666667, 0.01646090534979423]}
 
    
    
    
    

In [3]:
 def motifEnrichment(
    foreground, 
    background
):
        
     # Foreground dictionary
    new_dict_foreground = dict()
    new_dict_foreground_prob = dict()
    new_dict_foreground_final = dict()
    new_dict_foreground_binom = dict()
    key_list_f = list(foreground.keys())
    length_list_f = list()
    
    # Background dictionary
    new_dict_background = dict()
    new_dict_background_prob = dict()
    key_list_b = list(background.keys())
    length_list_b = list()
    
    # Foreground dictionary
    # Calulates sum of motifs of same length
    for i in key_list_f: 
        
        if len(i) in length_list_f:
            new_dict_foreground[len(i)] += foreground[i]     
        else: 
            new_dict_foreground[len(i)] = foreground[i]
            length_list_f.append(len(i))
            
    # Calculates probability of occurrence of motifs
    for i in new_dict_foreground:
        
        prob_of_occurrence_f = new_dict_foreground[i] / sum(foreground.values())
        new_dict_foreground_prob[i] = prob_of_occurrence_f
    
    # Background dictionary
    # Calulates sum of motifs of same length
    for i in key_list_b: 
        
        if len(i) in length_list_b:
            new_dict_background[len(i)] += background[i]     
        else: 
            new_dict_background[len(i)] = background[i]
            length_list_b.append(len(i))
    
    # Calculates probability of occurrence of motifs
    for i in new_dict_background:
        
        prob_of_occurrence_f = new_dict_background[i] / sum(background.values())
        new_dict_background_prob[i] = prob_of_occurrence_f    
    
    # Main
    # Calculates enrichment 
    enrichment_dict = dict()
    
    for i in new_dict_foreground_prob:

        enrichment_dict[i] = new_dict_foreground_prob[i] / new_dict_background_prob[i]    
    
    # Calculate p-value for foreground motifs    
    n = len(foreground)
    p = np.average(list(new_dict_background_prob.values()))
        
    for i in foreground:
        
        r = foreground[i]
        new_dict_foreground_binom[i] = (1-binom.cdf(r, n, p))
       
    # Create dictionary containing enrichment and p-values 
    for i in foreground: 

        new_dict_foreground_final[i] = [enrichment_dict[len(i)], new_dict_foreground_binom[i]]
    
    print(new_dict_foreground_final)
    return(new_dict_foreground_final)


In [4]:
motifEnrichment(FOREGROUND_DICT, BACKGROUND_DICT)

{'UGAUUC': [0.7333333333333334, 0.00137174211248281], 'UAAACC': [0.7333333333333334, 0.10013717421124824], 'AAGUUACCU': [1.375, 0.6488340192043895], 'AAGCCUU': [1.6041666666666667, 0.6488340192043895], 'AGUUCUA': [1.6041666666666667, 0.6488340192043895], 'UUUCCCG': [1.6041666666666667, 0.00137174211248281]}


{'UGAUUC': [0.7333333333333334, 0.00137174211248281],
 'UAAACC': [0.7333333333333334, 0.10013717421124824],
 'AAGUUACCU': [1.375, 0.6488340192043895],
 'AAGCCUU': [1.6041666666666667, 0.6488340192043895],
 'AGUUCUA': [1.6041666666666667, 0.6488340192043895],
 'UUUCCCG': [1.6041666666666667, 0.00137174211248281]}

Testing

In [17]:
"""Unit tests for motif_enrichment.py"""

import pytest
from htsinfer.htsinfer import motif_enrichment

FOREGROUND_DICT = {"UGAUUC": 5, "UAAACC": 3, "AAGUUACCU": 1, "AAGCCUU": 1, "AGUUCUA": 1,
                   "UUUCCCG": 5}
BACKGROUND_DICT = {"UGAUUC": 3, "UAAACC": 5, "AAGCCUUAU": 1, "AGUUCUA": 1, "UUUCCCG": 5,
                   "UUGGAA": 7}
FOREGROUND_STR = {"UGAUUC": "test", "UAAACC": 3, "AAGUUACCU": 1, "AAGCCUU": 1, "AGUUCUA": 1,
                  "UUUCCCG": 5}
FINAL_DICT = {'UGAUUC': [0.7333333333333334, 0.00137174211248281],
              'UAAACC': [0.7333333333333334, 0.10013717421124824],
              'AAGUUACCU': [1.375, 0.6488340192043895],
              'AAGCCUU': [1.6041666666666667, 0.6488340192043895],
              'AGUUCUA': [1.6041666666666667, 0.6488340192043895],
              'UUUCCCG': [1.6041666666666667, 0.00137174211248281]}


class TestMotifEnrichment:
    """Tests for 'motif_enrichment' function."""

    def test_no_dictionaries(self):
        "No arguments passed."
        with pytest.raises(TypeError):
            motif_enrichment()

    def test_one_dictionary(self):
        "Missing one required argument."
        with pytest.raises(TypeError):
            motif_enrichment(FOREGROUND_DICT)

    def test_str_as_value(self):
        "Unsupported operand type 'str' as value in dictionary."
        with pytest.raises(TypeError):
            motif_enrichment(FOREGROUND_STR, BACKGROUND_DICT)

    def test_valid_dictionaries(self):
        "Valid dictionaries passed."
        assert motif_enrichment(FOREGROUND_DICT, BACKGROUND_DICT) == FINAL_DICT


ImportError: cannot import name 'infer_single_paired' from partially initialized module 'htsinfer' (most likely due to a circular import) (C:\Users\ritpau00\OneDrive - Universität Basel\Studium\Master\M1\Programming for life sciences\htsinfer\htsinfer\htsinfer.py)